# Simulazione del Problema degli N-Corpi

Questa simulazione rappresenta un sistema di particelle che interagiscono gravitazionalmente secondo la legge di Newton. Ogni particella ha una massa, una posizione e una velocità iniziali generate casualmente. Il raggio delle particelle è proporzionale a $\sqrt{\text{massa}}$, per rappresentare visivamente la loro dimensione relativa.

Le interazioni gravitazionali sono calcolate utilizzando la legge di Newton:

$$
F = G \frac{m_1 m_2}{r^2}
$$

Dove:
- $F$ è la forza gravitazionale tra due particelle.
- $G$ è la costante gravitazionale.
- $m_1$ e $m_2$ sono le masse delle particelle.
- $r$ è la distanza tra i loro centri.

Ogni particella subisce l'influenza gravitazionale di tutte le altre, e la forza risultante modifica la sua velocità e posizione nel tempo. La velocità iniziale è influenzata dalla temperatura $T$, che determina l'ampiezza delle velocità casuali.

Il sistema evolve iterativamente, mostrando le traiettorie delle particelle e simulando dinamiche come quelle di un sistema planetario o di cluster di particelle interagenti.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../../risorse')

In [61]:
import random
from particles import Particle, NParticlesSystem

class NBody(NParticlesSystem):
    def __init__(self, n, T, width, height, G=1, mass_range=(1, 5), background_color="#000000"):
        particles = []

        x = width / 2
        y = height / 2
        vx = 0
        vy = 0
        mass = 1000
        radius = mass**0.5  # Calcola il raggio in base alla massa
        color = "#FFFFFF"
        central_attractor = Particle(x, y, vx, vy, radius, mass, color)
        particles.append(central_attractor)
        
        for _ in range(n):
            x = random.uniform(0, width)
            y = random.uniform(0, height)
            vx = random.uniform(-T**0.5, T**0.5)
            vy = random.uniform(-T**0.5, T**0.5)
            mass = random.uniform(*mass_range)
            radius = mass**0.5  # Calcola il raggio in base alla massa
            color = f"#{random.randint(0, 0xFFFFFF):06x}"  # Colore casuale
            particle = Particle(x, y, vx, vy, radius, mass, color)
            particles.append(particle)

        super().__init__(particles, background_color)
        self.G = G
        self.width = width
        self.height = height


    def evolve(self, dt):
        for i, p1 in enumerate(self.particles):
            fx, fy = 0, 0
            for j, p2 in enumerate(self.particles):
                if i != j:
                    # Compute the vector between particles
                    dx = p2.x - p1.x
                    dy = p2.y - p1.y
                    dist_sq = dx**2 + dy**2

                    # Avoid division by zero (overlapping particles)
                    if dist_sq == 0:
                        continue

                    # Compute gravitational force magnitude
                    force = min(self.G * p1.mass * p2.mass / dist_sq, 10)
                    distance = dist_sq**0.5

                    # Compute force components
                    fx += force * (dx / distance)
                    fy += force * (dy / distance)

            # Update velocity based on force
            p1.vx += fx / p1.mass * dt
            p1.vy += fy / p1.mass * dt

        # Update positions after all forces have been calculated
        for particle in self.particles:
            particle.move(dt)


In [63]:
from animation import Animation
from ipycanvas import Canvas

canvas = Canvas(width=400, height=400)

# Colori
BACKGROUND_COLOR = "#496780"

nbody = NBody(25, 0, canvas.width, canvas.height, G=1, mass_range=(10, 100), background_color=BACKGROUND_COLOR)

def update():
    nbody.update(1, canvas)

anim = Animation(canvas, update)
anim.start()
anim.display()